# Ранжирование

Будем ранжировать разные данные с помощью XGBoost. <br>
В XGBoost используется алгоритм LambdaMART, который осуществляет pairwise ранжирование.

## Пример

Взят [отсюда](https://github.com/dmlc/xgboost/blob/master/demo/rank/rank_sklearn.py)

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import ndcg_score

Данные: датасет LETOR 4.0 MQ2008 - бенчмарк для ранжирования.

Данные представлены так:
* один объект - это запрос, один документ к нему (набор каких-то признаков) и одна метка релевантности (target)
* соответственно, для одного и того же запроса может быть несколько объектов
* информация, что, например, какие-то пять объектов относятся к одному запросу, содержится в отдельной структуре "groups" и передается в обучение

Читаем объекты и таргеты:

In [ ]:
x_train, y_train = load_svmlight_file("data/mq2008.train")
x_valid, y_valid = load_svmlight_file("data/mq2008.vali")
x_test, y_test = load_svmlight_file("data/mq2008.test")

Посмотрим на данные:
* в обучении 9630 объектов
* 46 признаков
* релевантность оценивается по трехбалльной шкале

In [ ]:
print(x_train.shape)
x_train[0].todense()

(9630, 46)


matrix([[0.007477, 0.      , 1.      , 0.      , 0.00747 , 0.      ,
         0.      , 0.      , 0.      , 0.      , 0.471076, 0.      ,
         1.      , 0.      , 0.477541, 0.00512 , 0.      , 0.571429,
         0.      , 0.004806, 0.768561, 0.727734, 0.716277, 0.582061,
         0.      , 0.      , 0.      , 0.      , 0.780495, 0.962382,
         0.999274, 0.961524, 0.      , 0.      , 0.      , 0.      ,
         0.797056, 0.697327, 0.721953, 0.582568, 0.      , 0.      ,
         0.      , 0.      , 0.      , 0.007042]])

In [ ]:
set(y_train)

{0.0, 1.0, 2.0}

Читаем информацию о группах:

In [ ]:
group_train = []
with open("data/mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("data/mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

group_test = []
with open("data/mq2008.test.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_test.append(int(line.split("\n")[0]))

Как устроена информация о группах:
* количество групп отражает информацию о количестве запросов
* каждое число обозначает количество последовательных объектов, которые в эту группу объединяются
* из предыдущего пункта следует, что в X объекты нельзя перемешивать
* если просуммировать все числа в списке групп, получим число объектов из X

Для чего нужны группы? <br>
Для того, чтобы в обучении не сравнивать доки из разных групп (разных запросов) между собой.

In [ ]:
print(len(group_train), sum(group_train))
group_train[:10]

471 9630


[8, 8, 8, 8, 8, 16, 8, 118, 16, 8]

Обучаем модель. <br>
С помощью `eval_set` можем контролировать обучение, но это необязательный параметр, можно обучить и без валидации. <br>
В параметре `objective` можно задать три опции: `rank:ndcg`, `rank:pairwise`, `rank:map`. `ndcg` и `map` регулияруют попарный лосс с помощью подсчета соответствующих метрик.

In [ ]:
params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

[0]	eval_0-map:0.71552
[1]	eval_0-map:0.72606
[2]	eval_0-map:0.72795
[3]	eval_0-map:0.73352


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=1.0, gpu_id=-1,
          importance_type='gain', interaction_constraints='', learning_rate=0.1,
          max_delta_step=0, max_depth=6, min_child_weight=0.1, missing=nan,
          monotone_constraints='()', n_estimators=4, n_jobs=0,
          num_parallel_tree=1, objective='rank:ndcg', random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
          tree_method='exact', validate_parameters=1, verbosity=None)

Получим предсказание на тестовом сете:

In [ ]:
pred = model.predict(x_test)

Посчитаем качество:

In [ ]:
start_idx = 0
grouped_pred = []
grouped_target = []

for group_n in group_test:
    grouped_pred.append(pred[start_idx:start_idx+group_n])
    grouped_target.append(y_test[start_idx:start_idx+group_n])
    start_idx += group_n

In [ ]:
np.mean([
    ndcg_score([grouped_target[i]], [grouped_pred[i]])
    for i in range(len(grouped_target))
])

0.5052327963105946

# Дз
Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:
* факт выполнения задания :)
* корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
* итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
* креативность признаков
* аккуратность ноутбука

Дедлайн: 15 октября

#### TRAIN TEST

In [1]:
import re
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [4]:
%cd /content/gdrive/My Drive/Вышечка/4 курс/инфопоиск/семинар 5

/content/gdrive/My Drive/Вышечка/4 курс/инфопоиск/семинар 5


In [14]:
answers = pd.read_csv('answers_all_data.csv')
queries = pd.read_csv("queries_all_data.csv")

In [15]:
answers.head()

,Номер связки,Текст вопросов,Текст ответа,Тематика,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...
3,327,Могу я отказаться от вакцинации?\nВ каких случ...,Согласно приказу Министерства здравоохранения ...,ВАКЦИНАЦИЯ,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...
4,328,Безопасна ли вакцинация?\nОпасна ли вакцинация...,В соответствии с пунктами 1 и 2 статьи 12 Феде...,ВАКЦИНАЦИЯ,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...


In [16]:
queries.head()

,Текст вопроса,Номер связки\n,Тематика,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",с уважением \n\nПосле 15 августа 2020 года к н...,с уважением \n \n После 15 августа 2020 года...,с уважение вероника игоревич ильич после 15 ав...,с уважение после 15 август 2020 год к мы в пла...,с уважение после 15 август 2020 год к мы в пла...
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,"Здравствуйте! Проинформируйте, пожалуйста, нуж...","Здравствуйте ! Проинформируйте , пожалуйста , ...",здравствуйте проинформировать пожалуйста нужны...,здравствуйте проинформировать пожалуйста нужны...,здравствуйте проинформировать пожалуйста нужны...
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,"--\nДобрый день!\n Меня, 30.07.1989г.р., п...","- - \n Добрый день ! Меня , 30 . 07 . 1989г . ...",-- добрый день я сидельников андрей олегович 3...,- - добрый день я 30 07 1989г р посадить на ка...,-- добрый день я 30071989гра посадить на каран...
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",Добрый день.\nВ согласно постановлению необх...,Добрый день . В согласно постановлению необход...,добрый день в кемеровский область согласно пос...,добрый день в согласно постановление необходим...,добрый день в согласно постановление необходим...
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ","Здравствуйте, в моем городе введено ограничен...","Здравствуйте , в моем городе введено ограничен...",здравствуйте в мой город кострома ввести огран...,здравствуйте в мой город ввести ограничение на...,здравствуйте в мой город ввести ограничение на...


In [17]:
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [18]:
queries = queries.fillna(0)

In [19]:
for_queries = queries[['Номер связки\n', 'Текст вопроса',
                       'вопросы_препроцессинговые',
                       'natasha',	'deeppavlov']]

In [20]:
for_queries.rename(columns={'Номер связки\n': 'Номер связки',
                            'Текст вопроса': 'Текст вопросов',
                            'natasha': 'natasha',
                            'deeppavlov': 'deeppavlov'},
                   inplace=True)

In [21]:
for_answers = answers[['Номер связки', 'Текст вопросов',
                       'вопросы_препроцессинговые',
                       'natasha',	'deeppavlov']]

In [22]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 35

train_df, test = train_test_split(for_queries, test_size=0.3,
                               random_state=RANDOM_STATE)

In [23]:
train = pd.concat([for_answers, train_df])

In [24]:
train.head()

,Номер связки,Текст вопросов,вопросы_препроцессинговые,natasha,deeppavlov
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...
1,78.0,Где сделать вакцинацию от коронавируса?\nСущес...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...
2,326.0,Сколько стоит сделать вакцину от гриппа?\nМожн...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...
3,327.0,Могу я отказаться от вакцинации?\nВ каких случ...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...
4,328.0,Безопасна ли вакцинация?\nОпасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...


In [25]:
test.head()

,Номер связки,Текст вопросов,вопросы_препроцессинговые,natasha,deeppavlov
1333,308.0,"В сентябре, хочу поехать на личном транспорте ...",в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...
1986,308.0,ДОбрый день!\n\nя прилетел в воскресенье рано ...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...
2282,37.0,Добрый день! Нужна ли справка об отсутствии Ко...,добрый день нужный ли справка о отсутствие ков...,добрый день нужный ли справка о отсутствие при...,добрый день нужный ли справка о отсутствие при...
810,5.0,Здравствуйте. Я работаю врачем в поликлинике. ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...
2147,1.0,"\nЗдравствуйте!\nСкажите, пожалуйста, граждани...",здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...


#### Классификатор

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
# исключается случаи, когда нестроковый тип данных

train['Текст вопросов'] = train['Текст вопросов'].astype(str)
train['вопросы_препроцессинговые'] = train['вопросы_препроцессинговые'].astype(str)
train['natasha'] = train['natasha'].astype(str) 
train['deeppavlov'] = train['deeppavlov'].astype(str) 
test['Текст вопросов'] = test['Текст вопросов'].astype(str)
test['вопросы_препроцессинговые'] = test['вопросы_препроцессинговые'].astype(str)
test['natasha'] = test['natasha'].astype(str)
test['deeppavlov'] = test['deeppavlov'].astype(str)

##### TF-IDF + Log regression

In [29]:
def tfidf_clf_log(train, test, subset, nums):
    tfidf_vectorizer = TfidfVectorizer()
    X_train = tfidf_vectorizer.fit_transform(train[subset])
    X_test = tfidf_vectorizer.transform(test[subset])

    # лог регрессия
    model = LogisticRegression().fit(
    X_train, train[nums])

    y_pred = model.predict(X_test)
    
    print(classification_report(test[nums], y_pred))

    print(f'Accuracy: {accuracy_score(test[nums], y_pred)}')
    
    return accuracy_score(test[nums], y_pred)

In [30]:
just_text = tfidf_clf_log(train, test, 'Текст вопросов', 'Номер связки')

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.13      0.23      0.17        39
         3.0       0.00      0.00      0.00         7
         5.0       0.00      0.00      0.00         5
         6.0       0.53      0.46      0.49        93
        10.0       0.00      0.00      0.00         2
        12.0       0.75      0.38      0.51        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.56      0.16      0.25        56
        38.0       0.00      0.00      0.00         3
        45.0       0.00      0.00      0.00        12
        56.0       0.00      0.00      0.00         1
        57.0       0.00      0.00      0.00        10
        70.0       1.00      0.08      0.15        12
        74.0       0.00      0.00      0.00         1
        79.0       0.86    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
prep_text = tfidf_clf_log(train, test, 'вопросы_препроцессинговые', 'Номер связки')

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.29      0.64      0.40        39
         3.0       0.00      0.00      0.00         7
         5.0       0.00      0.00      0.00         5
         6.0       0.53      0.65      0.58        93
        10.0       0.00      0.00      0.00         2
        12.0       0.76      0.49      0.59        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.59      0.34      0.43        56
        38.0       0.00      0.00      0.00         3
        45.0       0.00      0.00      0.00        12
        56.0       0.00      0.00      0.00         1
        57.0       0.00      0.00      0.00        10
        70.0       0.91      0.83      0.87        12
        74.0       0.00      0.00      0.00         1
        79.0       0.88    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
natasha_text = tfidf_clf_log(train, test, 'natasha', 'Номер связки')

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.28      0.62      0.39        39
         3.0       0.00      0.00      0.00         7
         5.0       1.00      0.20      0.33         5
         6.0       0.51      0.58      0.55        93
        10.0       0.00      0.00      0.00         2
        12.0       0.77      0.51      0.62        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.47      0.25      0.33        56
        38.0       0.00      0.00      0.00         3
        45.0       0.00      0.00      0.00        12
        56.0       0.00      0.00      0.00         1
        57.0       0.00      0.00      0.00        10
        70.0       0.91      0.83      0.87        12
        74.0       0.00      0.00      0.00         1
        79.0       0.88    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
deeppavlov_text = tfidf_clf_log(train, test, 'deeppavlov', 'Номер связки')

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.27      0.59      0.37        39
         3.0       0.00      0.00      0.00         7
         5.0       1.00      0.20      0.33         5
         6.0       0.56      0.62      0.59        93
        10.0       0.00      0.00      0.00         2
        12.0       0.83      0.51      0.63        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.35      0.20      0.25        56
        38.0       0.00      0.00      0.00         3
        45.0       0.00      0.00      0.00        12
        56.0       0.00      0.00      0.00         1
        57.0       0.00      0.00      0.00        10
        70.0       0.91      0.83      0.87        12
        74.0       0.00      0.00      0.00         1
        79.0       0.78    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




---


Cамый хороший результат получается на лемматизированных текстах (самый высокий) и на тексте, прошедшим препроцессинг с NER. Таким образом, классификаторы, которые будут тестироваться дальше будут прогоняться через эти два параметра.

##### TF-IDF (н-граммы) + Log regression update


добавляем н-граммы + др. параметры

In [34]:
def tfidf_clf_log_2(train, test, subset, nums):
    tfidf_vectorizer = TfidfVectorizer(analyzer='char_wb',
                                        ngram_range=(2, 5),
                                        max_df=0.97,
                                        min_df=3)
    X_train = tfidf_vectorizer.fit_transform(train[subset])
    X_test = tfidf_vectorizer.transform(test[subset])

    # лог регрессия
    model = LogisticRegression(class_weight='balanced',
                               solver='liblinear').fit(
    X_train, train[nums])

    y_pred = model.predict(X_test)
    
    print(classification_report(test[nums], y_pred))

    print(f'Accuracy: {accuracy_score(test[nums], y_pred)}')

    return accuracy_score(test[nums], y_pred)

In [35]:
prep_text_2 = tfidf_clf_log_2(train, test, 'вопросы_препроцессинговые', 'Номер связки')

              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         3
         1.0       0.41      0.46      0.43        39
         3.0       0.29      0.29      0.29         7
         5.0       0.27      0.60      0.37         5
         6.0       0.74      0.37      0.49        93
        10.0       1.00      1.00      1.00         2
        12.0       0.76      0.72      0.74        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.86      0.43      0.57        56
        38.0       0.00      0.00      0.00         3
        43.0       0.00      0.00      0.00         0
        45.0       0.23      0.25      0.24        12
        46.0       0.00      0.00      0.00         0
        56.0       0.00      0.00      0.00         1
        57.0       0.40      1.00      0.57        10
        70.0       0.75    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
deeppavlov_text_2 = tfidf_clf_log_2(train, test, 'deeppavlov', 'Номер связки')

              precision    recall  f1-score   support

         0.0       0.50      0.67      0.57         3
         1.0       0.42      0.51      0.46        39
         3.0       0.20      0.29      0.24         7
         5.0       0.25      0.60      0.35         5
         6.0       0.68      0.34      0.46        93
        10.0       0.50      1.00      0.67         2
        12.0       0.74      0.67      0.70        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.69      0.20      0.31        56
        38.0       0.00      0.00      0.00         3
        43.0       0.00      0.00      0.00         0
        45.0       0.29      0.33      0.31        12
        56.0       0.00      0.00      0.00         1
        57.0       0.40      1.00      0.57        10
        70.0       0.80      1.00      0.89        12
        74.0       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Есть значительные улучшения. При добавлении н-грамм и подкручивании параметров логистической регрессии получилось:



*   тексты после препроцессинга:




In [37]:
print(prep_text, '-->', prep_text_2)

0.6405797101449275 --> 0.6797101449275362


*   тексты после препроцессинга с диппавловым: 

In [38]:
print(deeppavlov_text, '-->', deeppavlov_text_2)

0.6217391304347826 --> 0.6463768115942029


##### TF-IDF (н-граммы) + SVC

In [39]:
from sklearn.svm import SVC

In [40]:
 def tfidf_clf_svc(train, test, subset, nums):
    tfidf_vectorizer = TfidfVectorizer(analyzer='char_wb',
                                        ngram_range=(2, 5), max_df=0.97,
                                        min_df=3)
    X_train = tfidf_vectorizer.fit_transform(train[subset])
    X_test = tfidf_vectorizer.transform(test[subset])

    model = SVC(kernel='linear',
                decision_function_shape='ovo').fit(
                    X_train, train[nums])

    y_pred = model.predict(X_test)
    
    print(classification_report(test[nums], y_pred))

    print(f'Accuracy: {accuracy_score(test[nums], y_pred)}')
    
    return accuracy_score(test[nums], y_pred)   

In [48]:
prep_text_3 = tfidf_clf_svc(train, test, 'вопросы_препроцессинговые', 'Номер связки')

              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         3
         1.0       0.27      0.72      0.39        39
         3.0       0.00      0.00      0.00         7
         5.0       0.67      0.40      0.50         5
         6.0       0.60      0.62      0.61        93
        10.0       0.00      0.00      0.00         2
        12.0       0.86      0.64      0.74        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.59      0.61      0.60        56
        38.0       0.00      0.00      0.00         3
        45.0       1.00      0.17      0.29        12
        56.0       0.00      0.00      0.00         1
        57.0       0.50      0.10      0.17        10
        70.0       0.91      0.83      0.87        12
        74.0       0.00      0.00      0.00         1
        79.0       0.61    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
deeppavlov_text_3 = tfidf_clf_svc(train, test, 'deeppavlov', 'Номер связки')

              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         3
         1.0       0.25      0.72      0.37        39
         3.0       0.00      0.00      0.00         7
         5.0       0.67      0.40      0.50         5
         6.0       0.62      0.61      0.62        93
        10.0       0.00      0.00      0.00         2
        12.0       0.85      0.56      0.68        39
        21.0       0.00      0.00      0.00         2
        32.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        37.0       0.44      0.43      0.43        56
        38.0       0.00      0.00      0.00         3
        45.0       1.00      0.17      0.29        12
        56.0       0.00      0.00      0.00         1
        57.0       0.67      0.20      0.31        10
        70.0       0.85      0.92      0.88        12
        74.0       0.00      0.00      0.00         1
        79.0       0.65    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


После смены модели для классификатора данные улучшились еще больше:

(три подпункта: тф-айдф + лог регрессия, тф-айдф + лог регрессия улучшенное и тф-айдф + svc)



*   тексты после препроцессинга:



In [50]:
print(prep_text, '-->', prep_text_2, '-->', prep_text_3)

0.6405797101449275 --> 0.6797101449275362 --> 0.6942028985507246




*   тексты после препроцессинга с диппавловым: 


In [51]:
print(deeppavlov_text, '-->', deeppavlov_text_2, '-->', deeppavlov_text_3)

0.6217391304347826 --> 0.6463768115942029 --> 0.6666666666666666
